In [ ]:
import numpy as np
from scipy.io import wavfile

# ---------------------------------------------------
# 1) 12-Tone Just Intonation Mapping
# ---------------------------------------------------
# For a chord, we'll treat the *lowest note* as the "root."
# Then each other note is measured in semitones above that root
# (mod 12). We apply these "pure" intervals.

JI_12TONE_MAP = {
    0:  1/1,     # unison
    1:  16/15,
    2:  9/8,
    3:  6/5,
    4:  5/4,
    5:  4/3,
    6:  45/32,   # or 7/5 in some JI systems
    7:  3/2,
    8:  8/5,
    9:  5/3,
    10: 16/9,
    11: 15/8,
    12: 2/1      # octave
}

# ---------------------------------------------------
# 2) Note-Name Parsing -> MIDI
# ---------------------------------------------------
NOTES_OFFSET_FROM_C = {
    "C": 0,
    "C#": 1,  "Db": 1,
    "D": 2,
    "D#": 3,  "Eb": 3,
    "E": 4,
    "F": 5,
    "F#": 6,  "Gb": 6,
    "G": 7,
    "G#": 8,  "Ab": 8,
    "A": 9,
    "A#": 10, "Bb": 10,
    "B": 11
}

def note_name_to_midi(note_name):
    """
    Convert text like 'C4', 'G#3' to a MIDI note number (e.g. 60, 68, etc.).
    A4 (440Hz) is MIDI 69.
    """
    # Split out the letter/accidental from the octave (which may be 1 or 2 digits).
    i = 0
    while i < len(note_name) and not note_name[i].isdigit() and note_name[i] != '-':
        i += 1
    letter_part = note_name[:i]   # e.g. "C#", "Bb", "G"
    octave_str = note_name[i:]    # e.g. "4", "3"
    
    octave = int(octave_str)
    semitone_offset = NOTES_OFFSET_FROM_C[letter_part]
    
    # MIDI note formula: note C0 is MIDI 12. So for octave=4, C4 => 60.
    midi_note = (octave + 1)*12 + semitone_offset
    return midi_note

def midi_to_freq(midi_note):
    """Standard 12-TET formula for frequency of a MIDI note (A4=440Hz)."""
    return 440.0 * 2.0 ** ((midi_note - 69)/12.0)

# ---------------------------------------------------
# 3) Waveform Generators
# ---------------------------------------------------
SAMPLE_RATE = 44100

def generate_sine_wave(freq, duration, sample_rate=SAMPLE_RATE):
    """
    Generate a sine wave of given frequency, duration (sec). Returns float32 array.
    """
    t = np.linspace(0, duration, int(sample_rate*duration), endpoint=False)
    wave = 0.2 * np.sin(2 * np.pi * freq * t)  # amplitude scaled
    return wave.astype(np.float32)

def generate_saw_wave(freq, duration, sample_rate=SAMPLE_RATE):
    """
    Generate a naive sawtooth wave. Returns float32 array.
    """
    t = np.linspace(0, duration, int(sample_rate*duration), endpoint=False)
    # Simple saw formula: saw(t) = (1 - 2*(fracPartOf(freq*t + 0.5))) ...
    wave = 0.2 * (1.0 - 2.0 * ((freq * t) - np.floor(freq * t + 0.5)))
    return wave.astype(np.float32)

# ---------------------------------------------------
# 4) Define a chord sequence, using note-name strings
# ---------------------------------------------------
# Example: We'll do a short progression. 
# Each element is ( [notes...], duration_in_seconds ).
chord_sequence = [
    (["C4", "E4", "G4"], 2.0),  # C major in standard notation
    (["C4", "Eb4", "G4"], 2.0), # C minor
    (["A4", "C5", "E5"], 2.0),  # A minor
    (["G3", "B3", "D4"], 2.0)   # G major
]

# ---------------------------------------------------
# 5) Offline Render With Just Intonation Retuning
# ---------------------------------------------------
all_audio = np.array([], dtype=np.float32)

for chord_notes, chord_duration in chord_sequence:
    # 1) Identify the root as the *lowest* MIDI note in the chord
    midi_values = [note_name_to_midi(n) for n in chord_notes]
    root_midi = min(midi_values)
    
    # 2) Convert root to standard TET frequency
    root_freq = midi_to_freq(root_midi)
    
    # Create an empty buffer for this chord
    chord_buffer = np.zeros(int(SAMPLE_RATE * chord_duration), dtype=np.float32)
    
    # 3) For each note in the chord, calculate its semitone offset from root
    #    Then map to a JI ratio, and generate that frequency.
    for note_midi in midi_values:
        interval_semitones = note_midi - root_midi
        
        # Because we are using a 12-tone JI map, let's wrap big intervals mod 12:
        # e.g. if a note is 19 semitones above the root, we do 19 % 12 = 7 
        # (which is still "perfect fifth" shape). 
        # This is a simplification; you may wish to handle multiple octaves differently.
        interval_mod_12 = interval_semitones % 12
        
        # Lookup the ratio. If interval_mod_12 > 12, we can clamp or handle it. 
        # We'll just ensure we have 0..12 in the dict.
        if interval_mod_12 in JI_12TONE_MAP:
            ratio = JI_12TONE_MAP[interval_mod_12]
        else:
            # fallback - if out of range, treat as unison or skip
            ratio = 1.0
        
        # If the chord note is more than 12 semitones above root, 
        # we may want to multiply by 2^(octaves_above). For instance:
        octaves_above = (interval_semitones // 12)
        # So final ratio = (JI ratio) * 2^(octaves_above).
        ratio *= 2.0 ** octaves_above
        
        # Now the final frequency in JI
        note_freq = root_freq * ratio
        
        # 4) Generate a waveform
        wave = generate_sine_wave(note_freq, chord_duration, sample_rate=SAMPLE_RATE)
        # or use generate_saw_wave(note_freq, chord_duration)
        
        # 5) Sum into chord buffer
        chord_buffer += wave
    
    # Concatenate chord buffer onto the overall track
    all_audio = np.concatenate((all_audio, chord_buffer))

# ---------------------------------------------------
# 6) Write to WAV
# ---------------------------------------------------
wavfile.write("just_intonation_sequence.wav", SAMPLE_RATE, all_audio)
print("Wrote 'just_intonation_sequence.wav' at 44.1 kHz sample rate.")
